In [ ]:
# !pip install librosa

In [1]:
import pandas as pd
import numpy as np
import librosa

# Read the Feather file
df = pd.read_feather('CRSP_daily_data_for_project(Technical_Analysis).feather')
# df = df[(df['PERMNO']==10000) |( df['PERMNO']==10001)|(df['PERMNO']==10002)| (df['PERMNO']==93434)|(df['PERMNO']==93435)|(df['PERMNO']==93436)]


In [2]:
# df[df['PERMNO']==10000]

In [3]:
from datetime import datetime
df['date']=pd.to_datetime(df['date'], format='%Y-%m-%d')
# df['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

In [4]:
df.sort_values(by=['date','PERMNO'],inplace=True)

In [5]:
df['Ret'] = df.groupby('PERMNO')['Close'].apply(lambda x: x.shift(-1).pct_change()).reset_index(level=0, drop=True)
df

/var/folders/nq/svc2nf551n74zpynk96y1b7w0000gn/T/ipykernel_13135/382757337.py:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Ret'] = df.groupby('PERMNO')['Close'].apply(lambda x: x.shift(-1).pct_change()).reset_index(level=0, drop=True)


,PERMNO,date,Open,High,Low,Close,Volume,SHROUT,vwretx,ewretx,sprtrn,Ret
19071,10006,1969-01-02,NaN,63.25000,63.00000,63.125,3400.0,5641.0,0.001022,0.005752,0.000674,NaN
38825,10014,1969-01-02,NaN,16.25000,15.37500,15.375,21100.0,4109.0,0.001022,0.005752,0.000674,NaN
164917,10057,1969-01-02,NaN,28.37500,28.12500,28.250,1300.0,2038.0,0.001022,0.005752,0.000674,NaN
293514,10102,1969-01-02,NaN,34.12500,33.25000,33.500,12300.0,11036.0,0.001022,0.005752,0.000674,NaN
386105,10137,1969-01-02,NaN,24.62500,24.12500,24.500,2400.0,19292.0,0.001022,0.005752,0.000674,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
92907830,93423,2023-12-29,24.92000,25.25000,24.64440,25.080,1306363.0,83537.0,-0.004084,-0.007546,-0.002826,0.0
92918058,93426,2023-12-29,33.79000,34.50000,33.79000,34.070,94057.0,12510.0,-0.004084,-0.007546,-0.002826,0.0
92926793,93429,2023-12-29,177.85001,179.09000,177.44000,178.560,521913.0,105556.0,-0.004084,-0.007546,-0.002826,0.0
92933480,93434,2023-12-29,0.62000,0.71495,0.58500,0.700,169185.0,43039.0,-0.004084,-0.007546,-0.002826,0.0


In [6]:
df['Ret'].dropna(inplace=True)

In [7]:
df = df[~df['Ret'].isnull()].reset_index(drop=True)
df

,PERMNO,date,Open,High,Low,Close,Volume,SHROUT,vwretx,ewretx,sprtrn,Ret
0,10006,1969-01-03,NaN,63.75000,63.12500,63.500,3600.0,5641.0,0.000115,0.001501,0.000577,-0.007874
1,10014,1969-01-03,NaN,15.75000,15.00000,15.125,26500.0,4109.0,0.000115,0.001501,0.000577,0.016529
2,10057,1969-01-03,NaN,28.37500,27.87500,28.250,2600.0,2038.0,0.000115,0.001501,0.000577,0.000000
3,10102,1969-01-03,NaN,33.87500,33.37500,33.500,11300.0,11036.0,0.000115,0.001501,0.000577,-0.026119
4,10137,1969-01-03,NaN,24.75000,24.37500,24.750,3900.0,19292.0,0.000115,0.001501,0.000577,-0.015152
...,...,...,...,...,...,...,...,...,...,...,...,...
68992266,93423,2023-12-29,24.92000,25.25000,24.64440,25.080,1306363.0,83537.0,-0.004084,-0.007546,-0.002826,0.000000
68992267,93426,2023-12-29,33.79000,34.50000,33.79000,34.070,94057.0,12510.0,-0.004084,-0.007546,-0.002826,0.000000
68992268,93429,2023-12-29,177.85001,179.09000,177.44000,178.560,521913.0,105556.0,-0.004084,-0.007546,-0.002826,0.000000
68992269,93434,2023-12-29,0.62000,0.71495,0.58500,0.700,169185.0,43039.0,-0.004084,-0.007546,-0.002826,0.000000


In [8]:
df = df[df['date'].dt.is_month_end]

In [11]:
df.shape

(2303257, 12)

In [ ]:
df['date'].max()

In [12]:
# Function: Generate CQT data for each stock
def create_cqt_df(stock_df, n_bins=20):
    stock_returns = stock_df['vwretx'].values

    # Data preprocessing: Ensure all values are finite
    stock_returns = np.nan_to_num(stock_returns, nan=0.0, posinf=0.0, neginf=0.0)

    sr = 1
    fmin = 0.01  # Minimum frequency
    hop_length = 1
    cqt_result = librosa.cqt(stock_returns, n_bins=n_bins, sr=sr, hop_length=hop_length, fmin=fmin)
    cqt_result_db = librosa.amplitude_to_db(np.abs(cqt_result), ref=np.mean)

    cqt_df = pd.DataFrame(cqt_result_db.T, columns=[f'CQT_{i+1}' for i in range(cqt_result_db.shape[0])])
    cqt_df = cqt_df.loc[:len(stock_df)-1, :]  # Ensure the length matches the original data
    cqt = pd.concat([stock_df['date'], cqt_df], axis=1)
    return cqt

# Function: Create rolling windows and retain date indices
def create_rolling_windows_with_dates(data, cqt_window_size, date_to_index):
    windows = []
    date_indices = []
    for i in range(len(data) - cqt_window_size + 1):
        window = data.iloc[i:i + cqt_window_size, 1:].values  # Exclude the date column, keep only CQT data
        windows.append(window)
        start_date = data.iloc[i]['date']
        if start_date in date_to_index:
            date_indices.append(date_to_index[start_date])
    return np.array(windows), date_indices


In [13]:
def get_pre_processed_data(df, start_date, end_date, window_size=3):
    
    df_sub = df[(df['date']>=start_date) & (df['date']<end_date)]

    # Get the list of all stock codes
    stock_codes = set(df_sub['PERMNO'].unique())
    # stock_codes = set()

    cqt_window_size = window_size
    n_bins = 2  # Number of CQT bins

    # Get all unique dates
    unique_dates = df_sub['date'].unique()

    # drop the PERMNO for which data is not available on all unique_dates
    for _,r in df_sub.groupby(['date']):
        stock_codes.intersection_update(set(r['PERMNO'].to_list()))

    stock_codes = list(stock_codes)
    
    df_sub = df_sub[df_sub['PERMNO'].isin(stock_codes)]
    
    # Create the 4D array, initialized with NaN
    num_dates = len(unique_dates)
    max_shape = (len(stock_codes), num_dates - cqt_window_size + 1, cqt_window_size, n_bins)
    final_4d_array = np.full(max_shape, np.nan)

    # Create the date-to-index mapping
    date_to_index = {date: idx for idx, date in enumerate(unique_dates)}

    # Create the stock code-to-index mapping
    stock_code_to_index = {stock_code: idx for idx, stock_code in enumerate(stock_codes)}

    # Iterate over each stock and fill the 4D array
    for stock_code in stock_codes:
        stock_idx = stock_code_to_index[stock_code]
        stock_data = df_sub[df_sub['PERMNO'] == stock_code].reset_index(drop=True)
        
        # Generate CQT data
        cqt_data = create_cqt_df(stock_data, n_bins)

        # Create rolling windows and retain date indices
        rolling_windows_3d, date_indices = create_rolling_windows_with_dates(cqt_data, cqt_window_size, date_to_index)
        
        # Fill the 4D array
        for window_idx, (window_data, date_idx) in enumerate(zip(rolling_windows_3d, date_indices)):
            if window_data.shape == (cqt_window_size, n_bins):
                final_4d_array[stock_idx, date_idx, :, :] = window_data

    # df_sub = df_sub.sort_values(by=['PERMNO', 'date'])
    # df_sub['Ret'] = df_sub.groupby('PERMNO')['Close'].apply(lambda x: x.shift(-1).pct_change()).reset_index(level=0, drop=True)


    return final_4d_array, df_sub, stock_codes

In [14]:
df['date'].min()

Timestamp('1969-01-31 00:00:00')

In [15]:
df[df['date'].dt.year==2007]['date'].unique()

<DatetimeArray>
['2007-01-31 00:00:00', '2007-02-28 00:00:00', '2007-04-30 00:00:00',
 '2007-05-31 00:00:00', '2007-07-31 00:00:00', '2007-08-31 00:00:00',
 '2007-10-31 00:00:00', '2007-11-30 00:00:00', '2007-12-31 00:00:00']
Length: 9, dtype: datetime64[ns]

In [16]:
start_date = df[df['date'].dt.year==2000]['date'].min()
end_date = df[df['date'].dt.year==2023]['date'].max()

In [ ]:
# start_date = datetime.strptime("2000-01-01", "%Y-%m-%d")
# end_date = datetime.strptime("2001-01-01", "%Y-%m-%d")

In [17]:
import numpy as np
# final_4d_array, df_sub, stock_codes = get_pre_processed_data(df, start_date, end_date)

In [ ]:
# stock=0
# stock_data = final_4d_array[stock]
# ret = df_sub[df_sub.PERMNO == stock_codes[stock]]['Ret'].values[-stock_data.shape[0]:].reshape(-1, 1)

In [ ]:
# model = Sequential()
# model.add(LSTM(50, input_shape=(final_4d_array.shape[1], 1)))
# model.add(Dropout(0.2))
# model.add(Dense(1))
# model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# scaler = MinMaxScaler()
# stock_data = scaler.fit_transform(stock_data.reshape(stock_data.shape[0], stock_data.shape[1] * stock_data.shape[2]))

# # Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(stock_data, ret, test_size=0.2, shuffle=False)


In [18]:
# Apply LSTM model to the CQT data
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [19]:
def apply_lstm_model(final_4d_array, df_sub, stock_codes, date):
    model = Sequential()
    model.add(LSTM(5, input_shape=(final_4d_array.shape[1], 1)))
    # model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    # make an array to store all X_test
    X_test_array = None
    y_test_array = None
    X_train_array = None
    y_train_array = None

    # returns = np.array([])

    for stock in range(final_4d_array.shape[0]):
        stock_data = final_4d_array[stock]
        ret = df_sub[df_sub.PERMNO == stock_codes[stock]]['Ret'].values[-stock_data.shape[0]:].reshape(-1, 1)
        # returns = np.append(returns, ret)

        # Normalize the data
        scaler = MinMaxScaler()
        stock_data = scaler.fit_transform(stock_data.reshape(stock_data.shape[0], stock_data.shape[1] * stock_data.shape[2]))

        # Split the data into train and test sets
        # print(stock_data.shape, ret.shape)
        # X_train = stock_data[:10]
        # y_train = stock_data[:10]
        # X_test = ret[10:]
        # y_test = ret[10:]

        X_train, X_test, y_train, y_test = train_test_split(stock_data, ret, test_size=0.5, shuffle=False)

        if X_test_array is None:
            X_test_array = np.zeros((final_4d_array.shape[0], X_test.shape[0], X_test.shape[1]))
        if X_train_array is None:
            X_train_array = np.zeros((final_4d_array.shape[0], X_train.shape[0], X_train.shape[1]))
        
        if y_test_array is None:
            y_test_array = np.zeros((final_4d_array.shape[0], y_test.shape[0], y_test.shape[1]))
        if y_train_array is None:
            y_train_array = np.zeros((final_4d_array.shape[0], y_train.shape[0], y_train.shape[1]))
            
        X_test_array[stock] = X_test
        X_train_array[stock] = X_train
        y_test_array[stock] = y_test
        y_train_array[stock] = y_train

    X_train_array = X_train_array.reshape(X_train_array.shape[0] * X_train_array.shape[1], X_train_array.shape[2])
    y_train_array = y_train_array.reshape(y_train_array.shape[0] * y_train_array.shape[1], y_train_array.shape[2])
    # Train the model
    train_loss = model.fit(X_train_array, y_train_array, epochs=2, batch_size=32)
    
    print("Train loss:", train_loss.history['loss'][-1])


    all_predictions = {
        date.year: {}
    }
    
    for stock in range(final_4d_array.shape[0]):
        stock_data = final_4d_array[stock]
        stock_data = stock_data.reshape(stock_data.shape[0], stock_data.shape[1] * stock_data.shape[2])
        all_predictions[date.year][stock_codes[stock]] = model.predict(stock_data, verbose = 0).tolist()
        # save numpy array to disk
        # np.save(f'predictions/{date.year}/{date.date().strftime("%Y%m%d")}_{stock_codes[stock]}_predictions.npy', predictions)
    
    import pandas as pd
    pd.DataFrame(all_predictions).to_csv(f'predictions/{date.date().strftime("%Y%m%d")}_predictions.csv')
    # X_test_array = X_test_array.reshape(X_test_array.shape[0] * X_test_array.shape[1], X_test_array.shape[2])
    # y_test_array = y_test_array.reshape(y_test_array.shape[0] * y_test_array.shape[1], y_test_array.shape[2])
    # # print(X_test_array.shape)
    # test_loss = model.evaluate(X_test_array, y_test_array, batch_size=1)
    # print("Test loss:", test_loss)
    # print(model.summary())
    return model

In [20]:
# import pandas as pd
# pd.DataFrame({
#     "2000":{'a':[1,2,3],
#     'b':[1,2,3]},
#     "2001":{'a':[1,2,3],
#     'b':[1,2,3]},
# })

In [ ]:
# model = apply_lstm_model(final_4d_array, df_sub, stock_codes)
# model.summary()

In [ ]:
# save keras model
# model.save('lstm_model.h5')

In [21]:
from dateutil.relativedelta import relativedelta

In [22]:
def train_model_annually(df, start_date, end_date, window_size=2):
    # models = []
    # iterate between start_date and end_date with 1 year step size
    while start_date < end_date:
        next_year = start_date + relativedelta(years=1)
        print(start_date, next_year)
        if next_year > end_date:
            next_year = end_date
        final_4d_array, df_sub, stock_codes = get_pre_processed_data(df, start_date, next_year, window_size)
        print("Got pre-processed data")
        model = apply_lstm_model(final_4d_array, df_sub, stock_codes, start_date)
        model.save(f'models/lstm_model_{start_date.date().strftime("%Y%m%d")}.keras')
        start_date = next_year
        # models.append(model)
    # return models

In [23]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')

In [24]:
train_model_annually(df, start_date, end_date, window_size=2)

2000-01-31 00:00:00 2001-01-31 00:00:00
Got pre-processed data
Epoch 1/2
760/760 ━━━━━━━━━━━━━━━━━━━━ 1s 803us/step - loss: 0.0976
Epoch 2/2
760/760 ━━━━━━━━━━━━━━━━━━━━ 1s 763us/step - loss: 0.0058
Train loss: 0.03603481873869896
2001-01-31 00:00:00 2002-01-31 00:00:00
Got pre-processed data
Epoch 1/2
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 822us/step - loss: 0.0052
Epoch 2/2
709/709 ━━━━━━━━━━━━━━━━━━━━ 1s 790us/step - loss: 0.0034
Train loss: 0.005277159623801708
2002-01-31 00:00:00 2003-01-31 00:00:00
Got pre-processed data
Epoch 1/2
494/494 ━━━━━━━━━━━━━━━━━━━━ 1s 819us/step - loss: 0.0120
Epoch 2/2
494/494 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - loss: 0.0029
Train loss: 0.003223957261070609
2003-01-31 00:00:00 2004-01-31 00:00:00
Got pre-processed data
Epoch 1/2
618/618 ━━━━━━━━━━━━━━━━━━━━ 1s 933us/step - loss: 0.0016
Epoch 2/2
618/618 ━━━━━━━━━━━━━━━━━━━━ 1s 831us/step - loss: 0.0016
Train loss: 0.001627884921617806
2004-01-31 00:00:00 2005-01-31 00:00:00
Got pre-processed data
Epoch 1/2
